In [1]:
import torch
from torch import nn
import math
import pandas as pd

In [2]:
#返回bert输入序列的标记，和对应的段索引
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

# 1、PositionWiseFFN、add&norm

In [3]:
class PositionWiseFFN(nn.Module):
    """
        基于位置的前馈网络
            由两个线性层构成的单隐藏层MLP
            ffn_num_input：输入数据维度
            ffn_num_hiddens：隐藏单元数
            ffn_num_outputs：输出数据维度
        
    """
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [4]:
#测试位置前馈网络
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4)))

tensor([[[-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717],
         [-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717],
         [-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717]],

        [[-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717],
         [-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717],
         [-0.9164, -0.1418, -0.1113,  0.0348, -0.1607,  0.3698, -0.0735,
           0.1717]]], grad_fn=<ViewBackward0>)

In [5]:
"""
    层规范化和批量规范化的区别
        层规范化：是对一个样本的所有特征进行均值为0方差为1处理
        批量规范化：是对所有样本的一个特征维度进行均值为0方差为1处理
"""
ln = nn.LayerNorm(3)#参数如果写一个整数，默认最后一个维度
bn = nn.BatchNorm1d(3)
X = torch.tensor([[1,2,3], [4,5, 6]], dtype=torch.float32)
# 在训练模式下计算X的均值和方差
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.2247,  0.0000,  1.2247],
        [-1.2247,  0.0000,  1.2247]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000, -1.0000],
        [ 1.0000,  1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


In [6]:
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    #X为输入，Y为残差
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [7]:
#dropout 会按照概率将元素设置为0，同时会按照比例乘以每个元素。
m = nn.Dropout(p=0.5)
input = torch.randn(3, 4)
output = m(input)
input,output

(tensor([[-0.4153, -0.2826,  0.5940, -1.1493],
         [ 0.9298,  2.0060, -0.1587, -0.1438],
         [-0.0955,  0.2116,  0.6681,  0.2892]]),
 tensor([[-0.8306, -0.0000,  0.0000, -2.2986],
         [ 1.8595,  0.0000, -0.0000, -0.0000],
         [-0.0000,  0.4232,  1.3361,  0.0000]]))

In [8]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()#评估模式下，暂退失效
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4)))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]], grad_fn=<NativeLayerNormBackward0>)

# 2、缩放点积注意力

In [9]:
#@save
def sequence_mask(X, valid_len, value=0):
    """在序列中屏蔽不相关的项
            将X列元素中的无效元素置为0        
    """
    #获取X的列长度
    maxlen = X.size(1)
    #以列长度生成一个向量，然后扩充到1*2矩阵
    #将valid扩充到2*1的矩阵
    #通过广播执行 < 运算得到一个valid长度*X列长度的掩码矩阵。
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    #将X中列元素为false的置为0，剩下X的有效列元素。
    X[~mask] = value
    return X

X = torch.tensor([[1, 2, 3], [4, 5, 6]])
sequence_mask(X, torch.tensor([1, 2]))

tensor([[1, 0, 0],
        [4, 5, 0]])

In [10]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        #如果没有指定有效长度，则对每个样本元素执行softmax操作
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        #如果valid是1维，则它是指定每个批量中有效长度分别为多少。
        #所以需要将valid中的每个元素扩充到每个批量中的样本个数，意思是同一个批量中的样本有效长度都一样。
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            #如果是2维的，valid指定了每个样本的有效长度，所以将valid变成一列与每个样本对其即可。
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        #X把每个批量的样本合并在一起，然后按照valid的向量指定的每个样本的长度屏蔽掉
        X = sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        
        #最后将X又还原成批量的三维形式
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [11]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.6102, 0.3898, 0.0000, 0.0000],
         [0.6150, 0.3850, 0.0000, 0.0000]],

        [[0.3969, 0.3148, 0.2883, 0.0000],
         [0.3778, 0.2441, 0.3782, 0.0000]]])

In [12]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.2529, 0.3527, 0.3944, 0.0000]],

        [[0.4425, 0.5575, 0.0000, 0.0000],
         [0.2970, 0.2274, 0.2220, 0.2536]]])

In [13]:
#@save
class DotProductAttention(nn.Module):
    """缩放点积注意力
        求得每个查询关于所有keys的注意力权重，然后对value求加权平均和，求的查询的值。
    """
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        #每个批量中，有多少个查询，都和keys的转置相乘做点积，求得每个查询对于不同key的评分函数值。
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        #保留有效分数后，获得每个查询的注意力权重。
        self.attention_weights = masked_softmax(scores, valid_lens)
        #利用注意力权重获得加权平均值，得到查询的对应值。
        return torch.bmm(self.dropout(self.attention_weights), values)

In [14]:
queries = torch.normal(0, 1, (2, 1, 2))
keys = torch.ones((2, 10, 2))
# 一个1*10*4的三维矩阵，被repeat翻倍成2*10*4的矩阵
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(
    2, 1, 1)
valid_lens = torch.tensor([2, 6])
attention = DotProductAttention(dropout=0.5)
attention.eval()
#查询和键生成了一个2*1*10的权重矩阵，和value做批量矩阵乘法求的加权和，变成2*1*4的矩阵。得到每个查询生成的分数
attention(queries, keys, values, valid_lens)

tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])

# 3、多头注意力汇聚

In [15]:
#@save
def transpose_qkv(X, num_heads):
    """
        为了多注意力头的并行计算而变换形状
            将X的维度切断后，分给不同的头处理。
            
    """
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [16]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        """
            qkv本应该是分别对应不同的放射变化，映射成不同维度分别送入不同的头。
            为了并行这个维度统一设为num_hiddens/h，也就说分别映射成这个维度送入不同的头。
            但是利用线性层可以将输出设为num_hiddens，等价于将qkv分别映射到h个头。
        """
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        #Wo的行Po等于num_hiddens，表示q的维度也应该是num_hiddens，将经过注意力计算后生成的8个头的注意力值维度为num_hiddens/h，
        #按照列向量concat后成为num_hiddens维度，然后与Wo相乘，生成num_hiddens的最终输出。
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)#第二个nuhiddens是为了保证和qkv输入的维度一样，这是nlp规定。

    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values线性变换后的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过分割变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            """
                valid 会对应每个被分割的q设置生成有效长度。
            """
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数，
        # num_hiddens/num_heads)
        """
            相当于并行对h个头求注意力汇聚，
        """
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        #将每个批量中不同头的维度合并，然后进行仿射变换。
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [17]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [18]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])

# 4、transformer block

In [19]:
#@save
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

# 5、bert

In [20]:
#@save
class BERTEncoder(nn.Module):
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        #多少个词元就有多少种嵌入表示，调用需要传入词元的索引，才能查到维度
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        #bert中有多个transformer块
        for i in range(num_layers):
            self.blks.add_module(f"{i}", EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        #让每个批量上的序列的每个词元都加上位置嵌入。
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

In [21]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

In [22]:
tokens = torch.randint(0, vocab_size, (2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

# 6、预训练任务

## Masked Language Modeling 

In [23]:
#@save
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务
            通过多层感知机，利用bert表示，预测每句话中指定位置的词元。
    
    """
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    def forward(self, X, pred_positions):
        #需要预测词元的位置数量
        num_pred_positions = pred_positions.shape[1]
        #将所有位置排成一列
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # 假设batch_size=2，num_pred_positions=3
        # 那么batch_idx是np.array（[0,0,0,1,1,1]）
        # 为了将同一个批量中的预测词元，与刚才改成一列的位置对应起来。
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        #获得需要预测词元的bert表示
        masked_X = X[batch_idx, pred_positions]
        #因为变成了2维，所以需要按照批量变成3维。
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        #通过mlp预测得出对应位置的预测结果，但是词元的维度是词表大小，不是bert表示。
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

In [24]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

In [25]:
#预测词元的真实标签
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
#计算交叉熵损失好像能自动独热
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

## Next Sentence Prediction

In [26]:
#@save
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)

In [27]:
#将X展平成(batchsize,num_hiddens)，也就说是一个句子，不分词元了。
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

In [28]:
#计算交叉上损失
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

# 7、BERT最终模型

In [29]:
#@save
class BERTModel(nn.Module):
    """BERT模型
            将输入的序列变成bert表示，并可以预测指定位置的词元，还可以预测输入的句子对是否是下一句关系。
    """
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        #这个隐藏层是将输入的<cls>词元bert表示再输出，然后传给下一句预测模型的输入。
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat